In [4]:
import cv2
import numpy as np
import pywt
import numpy as np
from PIL import Image
import pywt
from scipy.linalg import svd, diagsvd, inv
from cryptography.fernet import Fernet
import random

In [53]:
import numpy as np
import cv2
from cryptography.fernet import Fernet

class SVDWatermarking:
    def __init__(self, host_image_path, watermark_image_path, scaling_factor=0.9, arnold_key=None, fernet_key=None):
        self.host_image = cv2.imread(host_image_path, cv2.IMREAD_GRAYSCALE)
        self.watermark_image = cv2.imread(watermark_image_path, cv2.IMREAD_GRAYSCALE)
        self.scaling_factor = scaling_factor
        self.arnold_key = arnold_key or 10
        self.fernet_key = fernet_key or Fernet.generate_key()
        self.watermark_dimensions = self.watermark_image.shape
        self.block_size = 8

    def arnold_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (x + y) % w
                    new_y = (x + 2 * y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        return result

    def arnold_inverse_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (2 * x - y) % w
                    new_y = (x - y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        return result

    def embed_watermark(self):
        # Preprocess watermark
        watermark = (self.watermark_image > 128).astype(int)
        watermark = self.arnold_transform(watermark, self.arnold_key)

        # Select embedding blocks using HVS
        blocks = [np.hsplit(row, self.host_image.shape[1] // self.block_size) for row in np.vsplit(self.host_image, self.host_image.shape[0] // self.block_size)]
        blocks = [block.astype(np.float32) for row in blocks for block in row]
        dct_blocks = [cv2.dct(block.reshape(-1, 8)).ravel() for block in blocks]
        selected_blocks_indices = np.argsort([np.sum(np.abs(dct_block)) for dct_block in dct_blocks])[:np.prod(self.watermark_dimensions)]
        selected_blocks = [blocks[idx] for idx in selected_blocks_indices]

        watermarked_image = np.copy(self.host_image)
        encrypted_coords = []
        for i, block in enumerate(selected_blocks):
            U, S, V = np.linalg.svd(block)
            watermark_bit = watermark.flat[i]
            if watermark_bit == 1:
                U[1, 0] = np.sign(U[1, 0]) * (np.abs(U[1, 0]) + self.scaling_factor)
                U[2, 0] = np.sign(U[2, 0]) * (np.abs(U[2, 0]) - self.scaling_factor)
            else:
                U[1, 0] = np.sign(U[1, 0]) * (np.abs(U[1, 0]) - self.scaling_factor)
                U[2, 0] = np.sign(U[2, 0]) * (np.abs(U[2, 0]) + self.scaling_factor)
            watermarked_block = np.dot(U, np.dot(np.diag(S), V))
            row, col = np.unravel_index(selected_blocks_indices[i], (self.host_image.shape[0] // self.block_size, self.host_image.shape[1] // self.block_size))
            watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size] = watermarked_block.astype(np.uint8)
            encrypted_coords.append(self.encrypt_coordinates(row, col))

        return watermarked_image, encrypted_coords
    def extract_watermark(self, watermarked_image, encrypted_coords):
        fernet = Fernet(self.fernet_key)
        watermark_bits = []
        for coord in encrypted_coords:
            row, col = self.decrypt_coordinates(coord, fernet)
            block = watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size]
            U, _, _ = np.linalg.svd(block)
            bit = self.extract_bit(U)
            watermark_bits.append(bit)

        watermark_image = np.array(watermark_bits).reshape(self.watermark_dimensions)
        watermark_image = self.arnold_inverse_transform(watermark_image, iterations=self.arnold_key)
        return watermark_image

    def extract_bit(self, U):
        if U[1, 0] > U[2, 0]:
            return 1
        else:
            return 0

    def encrypt_coordinates(self, row, col):
        fernet = Fernet(self.fernet_key)
        coords = f"{row},{col}".encode()
        return fernet.encrypt(coords)

    def decrypt_coordinates(self, encrypted_coords, fernet):
        decoded_coords = fernet.decrypt(encrypted_coords).decode()
        row, col = map(int, decoded_coords.split(","))
        return row, col

In [54]:
arnold_key = 10 # Key for Arnold transform
fernet_key = Fernet.generate_key() # Symmetric encryption key for coordinates

watermarker = SVDWatermarking('/Users/kabir/Downloads/Watermark/images/1000.PNG', '/Users/kabir/Downloads/Watermark/images/dice_water.png',arnold_key=arnold_key, fernet_key=fernet_key)
watermarked_image, encrypted_coords = watermarker.embed_watermark()
cv2.imwrite('/watermarked_image_new.png', watermarked_image)

extracted_watermark = watermarker.extract_watermark(watermarked_image,encrypted_coords)
# cv2.imwrite('/extracted_watermark_new.png', extracted_watermark * 255)

In [55]:
# Ensure that the values are within the 0-255 range for uint8
normalized_watermark = np.clip(extracted_watermark * 255, 0, 255)

# Convert the data type to uint8
watermark_uint8 = normalized_watermark.astype(np.uint8)

# Create the image
image = Image.fromarray(watermark_uint8)
image.show()

In [49]:
image = Image.fromarray(watermarked_image)
image.show()

In [168]:
import numpy as np
import cv2
import pywt
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from base64 import urlsafe_b64encode
import os

class SVDWatermarking:
    def __init__(self, host_image_path, watermark_image_path, scaling_factor=0.7, arnold_key=None, key=None):
        self.host_image = cv2.imread(host_image_path, cv2.IMREAD_GRAYSCALE)
        self.watermark_image = cv2.imread(watermark_image_path, cv2.IMREAD_GRAYSCALE)
        self.scaling_factor = scaling_factor
        self.arnold_key = arnold_key or 1
        self.key = key or os.urandom(24)
        self.watermark_dimensions = self.watermark_image.shape
        self.block_size = 8
        self.iv = os.urandom(16)  # Initialization vector for AES


    def arnold_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (x + y) % w
                    new_y = (x + 2 * y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        print(str(result)+" transform ")
        return result

    def arnold_inverse_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (2 * x - y) % w
                    new_y = (y - x) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        print(str(result)+" inverse ")
        return result

    def embed_watermark(self):
        watermark = (self.watermark_image > 128).astype(int)
        watermark = self.arnold_transform(watermark, self.arnold_key)
        blocks = [np.hsplit(row, self.host_image.shape[1] // self.block_size) for row in np.vsplit(self.host_image, self.host_image.shape[0] // self.block_size)]
        blocks = [block.astype(np.float32) for row in blocks for block in row]
        selected_blocks_indices = np.argsort([np.sum(np.abs(block)) for block in blocks])[:np.prod(self.watermark_dimensions)]
        print(selected_blocks_indices)
        selected_blocks = [blocks[idx] for idx in selected_blocks_indices]

        watermarked_image = np.copy(self.host_image)
        encrypted_coords = []
        backend = default_backend()
        key = urlsafe_b64encode(self.key)
        cipher = Cipher(algorithms.AES(key), modes.CBC(self.iv), backend=backend)
        encryptor = cipher.encryptor()
        padder = padding.PKCS7(algorithms.AES.block_size).padder()


        for i, block in enumerate(selected_blocks):
        # Apply DWT and perform SVD
            coeffs = pywt.dwt2(block, 'haar')
            LL, (LH, HL, HH) = coeffs
            U, S, V = np.linalg.svd(LL, full_matrices=False)
            watermark_bit = watermark.flat[i]
            mu = (abs(U[2, 1]) + abs(U[3, 1])) / 2

            if i % 2 == 0:  # Even index, modify U
                U[2, 1] = np.sign(U[2, 1]) * (mu + (self.scaling_factor if watermark_bit else -self.scaling_factor))
                U[3, 1] = np.sign(U[3, 1]) * (mu - (self.scaling_factor if watermark_bit else self.scaling_factor))
            else:  # Odd index, modify V
                V[2, 1] = np.sign(V[2, 1]) * (mu + (self.scaling_factor if watermark_bit else -self.scaling_factor))
                V[3, 1] = np.sign(V[3, 1]) * (mu - (self.scaling_factor if watermark_bit else self.scaling_factor))

            LL_new = np.dot(U, np.dot(np.diag(S), V))
            coeffs_new = (LL_new, (LH, HL, HH))
            block_new = pywt.idwt2(coeffs_new, 'haar')

            row, col = np.unravel_index(selected_blocks_indices[i], (self.host_image.shape[0] // self.block_size, self.host_image.shape[1] // self.block_size))
            watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size] = block_new.astype(np.uint8)

            # Encrypt coordinates, re-initialize padder and encryptor each iteration
            padder = padding.PKCS7(algorithms.AES.block_size).padder()
            encryptor = cipher.encryptor()  # Reinitialize encryptor for each block

            coords = f"{row},{col}".encode()
            padded_data = padder.update(coords) + padder.finalize()
            encrypted_coords.append(encryptor.update(padded_data) + encryptor.finalize())
        return watermarked_image, encrypted_coords

    def extract_watermark(self, watermarked_image, encrypted_coords):
        decrypted_coords = []
        backend = default_backend()
        key = urlsafe_b64encode(self.key)
        cipher = Cipher(algorithms.AES(key), modes.CBC(self.iv), backend=backend)
        decryptor = cipher.decryptor()
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        watermark_bits = []

        for coord in encrypted_coords:
            # Decrypt coordinates, re-initialize unpadder and decryptor each iteration
            unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
            decryptor = cipher.decryptor()  # Reinitialize decryptor for each block

            decrypted_data = decryptor.update(coord) + decryptor.finalize()
            decrypted_coords.append(unpadder.update(decrypted_data) + unpadder.finalize())
            row, col = map(int, decrypted_coords[-1].decode().split(','))

            block = watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size]
            coeffs = pywt.dwt2(block, 'haar')
            LL, _ = coeffs
            U, _, _ = np.linalg.svd(LL, full_matrices=False)

            bit = 1 if U[2, 1] > U[3, 1] else 0
            watermark_bits.append(bit)


        watermark_image = np.array(watermark_bits).reshape(self.watermark_dimensions)
        watermark_image = self.arnold_inverse_transform(watermark_image, self.arnold_key)
        return watermark_image

In [169]:
arnold_key = 1 # Key for Arnold transform
watermarker = SVDWatermarking('/Users/kabir/Downloads/Watermark/images/1000.PNG', '/Users/kabir/Downloads/Watermark/images/dice_water.png',arnold_key=arnold_key)
watermarked_image, encrypted_coords = watermarker.embed_watermark()

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]] transform 
[3537 2407 2406 ... 1697 3864 1176]


In [170]:
print(encrypted_coords)

[b'\xae.O\x11\xc8>\xf2\x9b\xe4|\x8bb\xc1b\xfd\x07', b'\xfd\xd6\xaf\x8f\x9d\xac\xd6HT\x04|\xc4\x87\x9bjf', b'\x8a\xfb\x86\x07\x91\xc7MU\xab\xd1\n\x91\xd0D\x13\xe8', b'\xc0\x0c\xb3A\xdd\xda\x9f\xf2Q\x15\xaes\x1a\nM%', b'm\xde\xc0\x85\x18\xc1\x07F\xa2\x84\xef\xa1\x92\xe3\x05\x87', b'\xdc\x96\x8b\x1f^KtQ)\x8e\x00\xc7\xb3\x16\xce]', b'v6\x94Y\xb5\xa6\xa9\\&Q}A\xd1\xac\x84\xba', b"'p~\xac\xe0\xb7q\xeeHP\xabM\xabgB\x9e", b'\nnv=-\xf8\xed\x8a\xc7\x90\xb95P\xc7\xe0\xb1', b'\xe3\x88\xc0\x07N54\xca\xde\x98\x89\x13\xf6P\xcb}', b'\x98\x02\x9b\x8dHT\xe5\xeeZ\xa56\t\xfc|\xa2\x8b', b'9\x92\x96\xd7\xf1K@\xbd\x10\x85\xcaQ.>\x93\x7f', b'@\xc9\x86\x02\xbf-\xd3\xf0\xd0,\x9d\xa1\xea\x8a^\xbf', b'\x05\x11_\xd2\xa0Y\xca\x92\x16\xc7\xf6\xc9\xe0\xaf2\xd5', b'`\x98\xf72|\x85\x08\x8c\x01\x13\x88\xc4G\xacS\x1c', b'_\x10\xe2\x033\x93\x8e\x01\xccV\x12\xb3\x8aO\xa9?', b'\xd5\xcc\xb6I6\xd5P\xee\xe2\xb4u\xecTmt\x19', b'\xad\x8d;\xf2\x83\x93\xb6\x1c\xf1\xfe\xd7\xfa\xf10\x14K', b'5X\xdaDd\xdar4U\xb1l\x9f\xaf3\x08\x18', b

In [171]:
image = Image.fromarray(watermarked_image)
image.show()

In [165]:
extracted_watermark = watermarker.extract_watermark(watermarked_image,encrypted_coords)


In [166]:
extracted_watermark

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 1, 1]])

In [167]:
# Ensure that the values are within the 0-255 range for uint8
normalized_watermark = np.clip(extracted_watermark * 255, 0, 255)

# Convert the data type to uint8
watermark_uint8 = normalized_watermark.astype(np.uint8)

# Create the image
image = Image.fromarray(watermark_uint8)
image.show()

In [133]:
def arnold_transform(image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (x + y) % w
                    new_y = (x + 2 * y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        print(str(result)+" transform ")
        return result

def arnold_inverse_transform(image, iterations):
    result = np.copy(image)
    h, w = image.shape
    for _ in range(iterations):
        for x in range(w):
            for y in range(h):
                new_x = (2 * x - y) % w
                new_y = (x - y) % h
                result[new_y, new_x] = image[y, x]
        image = np.copy(result)
    print(str(result)+" inverse ")
    return result


In [134]:
host_image = cv2.imread('/Users/kabir/Downloads/Watermark/images/1000.PNG', cv2.IMREAD_GRAYSCALE)

In [135]:
at=arnold_transform(image=host_image,iterations=10)

[[123   0 202 ... 198 193 175]
 [163  39   0 ...   0 194 205]
 [  0 146  21 ...  21  28 203]
 ...
 [204 160 163 ... 167 207  21]
 [  0 189 169 ... 182 160 212]
 [ 12   0 170 ... 187 203 145]] transform 


In [137]:
image = Image.fromarray(at)
image.show()
image.save('at.png')

In [145]:
att_image = cv2.imread('/Users/kabir/Downloads/Watermark/exps/at.png', cv2.IMREAD_GRAYSCALE)

In [152]:
ctt=arnold_inverse_transform(image=host_image ,iterations=10)

[[123 202 200 ... 175 155 140]
 [169 149 132 ... 208 198 184]
 [205 192 179 ...  82 101  63]
 ...
 [166 156   0 ... 191 185 176]
 [190 182 172 ... 199 198 194]
 [199 196 193 ... 129 203 200]] inverse 


In [153]:
image = Image.fromarray(ctt)
image.show()
image.save('att.png')